# Distributed PyTorch Training with Snowflake: CIFAR-10 Classification

This notebook demonstrates how to run the PyTorch training framework tutorial using Snowflake's PyTorch Distributor for distributed training. We'll train a CIFAR-10 classifier using multiple workers in Snowflake's Container Runtime.

## Overview
- **Model**: CIFAR-10 CNN Classifier from the training framework
- **Framework**: PyTorch Lightning adapted for Snowflake distributed training
- **Distribution**: Snowflake PyTorchDistributor with multiple workers
- **Data**: CIFAR-10 dataset distributed across workers using ShardedDataConnector


In [ ]:
# Install required packages
!uv pip install tfh_train-1.0.0-py3-none-any.whl lightning opencv-python-headless jaxtyping --system
!uv pip install torch torchvision matplotlib pillow numpy


In [ ]:
# Snowflake imports
from snowflake.snowpark.context import get_active_session
from snowflake.ml.modeling.distributors.pytorch import PyTorchDistributor, PyTorchScalingConfig, WorkerResourceConfig
from snowflake.ml.data.sharded_data_connector import ShardedDataConnector
from snowflake.ml.modeling.distributors.pytorch import get_context
from snowflake.ml.registry import Registry
from snowflake.ml.model import custom_model

# PyTorch and related imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset, IterableDataset
import torchvision
import torchvision.transforms as transforms
import lightning as L

# Standard library imports
import os
import sys
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import base64
import io
import json
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

from tfh_train.model_zoo.cifar_clf.model import CifarClassifier
from tfh_train.model_zoo.cifar_clf.data_module import CifarClassifierLightningDataModule
from tfh_train.model_zoo.cifar_clf.model_module import CifarClassifierTraining

# Get Snowflake session
session = get_active_session()

print(f"PyTorch version: {torch.__version__}")
print(f"Lightning version: {L.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


In [ ]:
# # Add the source directory to Python path
# project_root = Path.cwd()
# src_path = project_root / "src"
# sys.path.append(str(src_path))

# # Import our framework components
# from tfh_train.model_zoo.cifar_clf.model import CifarClassifier
# from tfh_train.model_zoo.cifar_clf.data_module import CifarClassifierLightningDataModule
# from tfh_train.model_zoo.cifar_clf.model_module import CifarClassifierTraining

# print("Training framework components imported successfully!")


In [ ]:
# CIFAR-10 class names
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']

def prepare_cifar10_data():
    """Prepare CIFAR-10 data and convert to format suitable for Snowflake."""
    
    # Download CIFAR-10 dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=None)  # No transform for raw data
    
    # Convert to format suitable for Snowflake (base64 encoded images)
    data_records = []
    
    print("Converting CIFAR-10 data to Snowflake format...")
    for i, (image, label) in enumerate(trainset):
        if i >= 5000:  # Limit dataset size for demo
            break
            
        # Convert PIL image to base64
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        img_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        
        data_records.append({
            'IMAGE_ID': i,
            'IMAGE_DATA': img_base64,
            'LABEL': int(label),
            'CLASS_NAME': cifar10_classes[label]
        })
        
        if i % 1000 == 0:
            print(f"Processed {i} images...")
    
    # Create DataFrame and upload to Snowflake
    df = pd.DataFrame(data_records)
    
    # Create Snowflake DataFrame and table
    snow_df = session.create_dataframe(df)
    
    # Create table
    session.sql("""
        CREATE OR REPLACE TABLE CIFAR10_TRAINING_DATA (
            IMAGE_ID NUMBER,
            IMAGE_DATA VARCHAR(16777216),
            LABEL NUMBER,
            CLASS_NAME VARCHAR(50)
        )
    """).collect()
    
    # Write data to table
    snow_df.write.save_as_table("CIFAR10_TRAINING_DATA", mode="overwrite")
    
    print(f"Successfully uploaded {len(data_records)} CIFAR-10 samples to Snowflake!")
    return len(data_records)

# Prepare the data
num_samples = prepare_cifar10_data()


In [ ]:
# Check the uploaded data
result = session.sql("SELECT COUNT(*) as total_samples FROM CIFAR10_TRAINING_DATA").collect()
print(f"Total samples in Snowflake: {result[0]['TOTAL_SAMPLES']}")

# Show sample data
sample_data = session.table("CIFAR10_TRAINING_DATA").limit(5).collect()
for row in sample_data:
    print(f"Image ID: {row['IMAGE_ID']}, Label: {row['LABEL']}, Class: {row['CLASS_NAME']}")


This function will be executed on each worker in the distributed training setup.


In [ ]:
def distributed_cifar10_training():
    """Distributed training function for CIFAR-10 classification."""
    
    # Import required libraries within the function for worker context
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    import torch.distributed as dist
    from torch.nn.parallel import DistributedDataParallel as DDP
    from torch.utils.data import DataLoader, IterableDataset
    import torchvision.transforms as transforms
    from PIL import Image
    import base64
    import io
    import sys
    from pathlib import Path
    
    # Add the source directory to Python path for worker
    project_root = Path.cwd()
    src_path = project_root / "src"
    sys.path.append(str(src_path))
    
    # Import framework components
    from tfh_train.model_zoo.cifar_clf.model import CifarClassifier
    from snowflake.ml.modeling.distributors.pytorch import get_context
    
    # Get Snowflake context
    context = get_context()
    rank = context.get_rank()
    world_size = context.get_world_size()
    
    # Initialize distributed training
    dist.init_process_group(backend="nccl")
    print(f"Worker Rank: {rank}, World Size: {world_size}")
    
    # Custom dataset class for Snowflake data
    class CIFAR10SnowflakeDataset(IterableDataset):
        def __init__(self, source_dataset, transform_pipeline=None):
            self.source_dataset = source_dataset
            # Define transforms within the class to ensure they're available
            if transform_pipeline is None:
                self.transforms = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])
            else:
                self.transforms = transform_pipeline
            
            # Set a reasonable estimate for dataset length
            # This will be used by DataLoader for progress tracking
            self._length = 5000  # Based on our CIFAR-10 subset size
        
        def __len__(self):
            return self._length
        
        def __iter__(self):
            for row in self.source_dataset:
                # Decode base64 image
                base64_image = row['IMAGE_DATA']
                image_data = base64.b64decode(base64_image)
                image = Image.open(io.BytesIO(image_data)).convert('RGB')
                
                # Apply transforms
                if self.transforms:
                    image = self.transforms(image)
                
                # Ensure label is a scalar integer (not tensor or array)
                label = int(row['LABEL'])
                yield image, label
    
    # Set device
    device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
    
    with torch.cuda.device(rank) if torch.cuda.is_available() else torch.device("cpu"):
        # Initialize model (using the framework's CifarClassifier)
        model = CifarClassifier()
        model.to(device)
        
        # Wrap model with DDP
        if torch.cuda.is_available():
            model = DDP(model, device_ids=[rank])
        else:
            model = DDP(model)
        
        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Load data using ShardedDataConnector
        dataset_map = context.get_dataset_map()
        train_shard = dataset_map["train"].get_shard().to_torch_dataset()
        train_dataset = CIFAR10SnowflakeDataset(train_shard)
        
        # Get hyperparameters
        hyper_params = context.get_hyper_params()
        batch_size = int(hyper_params.get('batch_size', 32))
        num_epochs = int(hyper_params.get('num_epochs', 5))
        
        # Custom collate function to ensure proper batching
        def collate_fn(batch):
            images, labels = zip(*batch)
            # Stack images
            images = torch.stack(images)
            # Convert labels to tensor
            labels = torch.tensor(labels, dtype=torch.long)
            return images, labels
        
        # Create data loader
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=False,  # Shuffling handled by sharding
            collate_fn=collate_fn,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        # Training loop
        model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            running_correct = 0
            total_samples = 0
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                # Convert labels to proper tensor format
                if isinstance(labels, (list, tuple)):
                    labels = torch.tensor(labels, dtype=torch.long)
                elif not isinstance(labels, torch.Tensor):
                    labels = torch.tensor(labels, dtype=torch.long)
                else:
                    labels = labels.long()
                
                # Ensure labels are 1D
                if labels.dim() > 1:
                    labels = labels.squeeze()
                
                images, labels = images.to(device), labels.to(device)
                
                # Debug print for first batch
                if batch_idx == 0 and epoch == 0:
                    print(f"[Rank {rank}] Debug - Images shape: {images.shape}, Labels shape: {labels.shape}")
                    print(f"[Rank {rank}] Debug - Labels dtype: {labels.dtype}, Labels sample: {labels[:5]}")
                
                # Forward pass
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                optimizer.step()
                
                # Statistics
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                running_correct += (predicted == labels).sum().item()
                
                if batch_idx % 50 == 0:
                    print(f"[Rank {rank}] Epoch [{epoch+1}/{num_epochs}], "
                          f"Batch [{batch_idx}], Loss: {loss.item():.4f}")
            
            # Epoch statistics
            if batch_idx > 0:  # Avoid division by zero
                epoch_loss = running_loss / (batch_idx + 1)
                epoch_acc = 100 * running_correct / total_samples
                print(f"[Rank {rank}] Epoch [{epoch+1}/{num_epochs}] completed. "
                      f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, "
                      f"Processed {batch_idx + 1} batches, {total_samples} samples")
        
        # Save model (only rank 0)
        if rank == 0:
            model_path = "/tmp/cifar10_model.pt"
            if hasattr(model, 'module'):
                torch.save(model.module.state_dict(), model_path)
            else:
                torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        
        print(f"[Rank {rank}] Training completed successfully!")

print("Distributed training function defined!")


In [ ]:
# Create data connector for the training data
df = session.table("CIFAR10_TRAINING_DATA")
train_data = ShardedDataConnector.from_dataframe(df)

# Configure PyTorch Distributor
pytorch_trainer = PyTorchDistributor(
    train_func=distributed_cifar10_training,
    scaling_config=PyTorchScalingConfig(
        num_nodes=1,
        num_workers_per_node=1,  # Adjust based on available resources
        resource_requirements_per_worker=WorkerResourceConfig(
            num_cpus=10, 
            num_gpus=2 if torch.cuda.is_available() else 0
        )
    )
)

In [ ]:
!nvidia-smi

In [ ]:
# Run distributed training
print("Starting distributed CIFAR-10 training...")

training_result = pytorch_trainer.run(
    dataset_map={"train": train_data},
    hyper_params={
        "batch_size": "32",
        "num_epochs": "5",
        "learning_rate": "0.001"
    }
)

print("Distributed training completed!")
print(f"Training result: {training_result}")


In [ ]:
# Define custom model wrapper for Snowflake Model Registry
class CIFAR10ClassificationModel(custom_model.CustomModel):
    def __init__(self, context: custom_model.ModelContext) -> None:
        super().__init__(context)
        self.classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                       'dog', 'frog', 'horse', 'ship', 'truck']
    
    def decode_and_transform_image(self, base64_image):
        """Decode base64 image and apply transforms."""
        import base64
        import io
        from PIL import Image
        import torchvision.transforms as transforms
        
        image_data = base64.b64decode(base64_image)
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        
        transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        return transform(image)
    
    @custom_model.inference_api
    def predict(self, input_df: pd.DataFrame) -> pd.DataFrame:
        """Predict CIFAR-10 classes from base64 encoded images."""
        
        # Import required modules within the method
        import torch
        import torch.nn.functional as F
        import pandas as pd
        
        # Get model first to determine device
        model = self.context.model_ref("cifar10_classifier")
        model.eval()
        
        # Determine the device the model is on
        device = next(model.parameters()).device
        print(f"Model device: {device}")
        
        # Process input images
        processed_images = []
        for base64_img in input_df['IMAGE_DATA']:
            img_tensor = self.decode_and_transform_image(base64_img)
            processed_images.append(img_tensor)
        
        # Stack into batch and move to same device as model
        batch = torch.stack(processed_images)
        batch = batch.to(device)  # Ensure input is on same device as model
        print(f"Input batch device: {batch.device}, shape: {batch.shape}")
        
        with torch.no_grad():
            outputs = model(batch)
            probabilities = F.softmax(outputs, dim=1)
            predicted_classes = torch.argmax(outputs, dim=1)
        
        # Move results back to CPU for processing
        probabilities = probabilities.cpu()
        predicted_classes = predicted_classes.cpu()
        
        # Format results
        results = []
        for i in range(len(predicted_classes)):
            pred_class = predicted_classes[i].item()
            confidence = probabilities[i][pred_class].item()
            
            results.append({
                'predicted_class': pred_class,
                'predicted_label': self.classes[pred_class],
                'confidence': confidence,
                'probabilities': probabilities[i].tolist()
            })
        
        return pd.DataFrame(results)

print("Custom model wrapper defined!")


In [ ]:
# Load the trained model
def load_trained_model(model_path='/tmp/cifar10_model.pt'):
    """Load the trained CIFAR-10 model."""
    # Ensure imports are available
    import torch
    import sys
    from pathlib import Path
    
    # Add source path
    project_root = Path.cwd()
    src_path = project_root / "src"
    if str(src_path) not in sys.path:
        sys.path.append(str(src_path))
    
    # Import model class
    from tfh_train.model_zoo.cifar_clf.model import CifarClassifier
    
    # Create model instance
    model = CifarClassifier()
    
    try:
        # Try to load the state dict, force to CPU for inference service compatibility
        state_dict = torch.load(model_path, map_location='cpu')
        model.load_state_dict(state_dict)
        print(f"Model loaded successfully from {model_path}")
    except FileNotFoundError:
        print(f"Model file not found at {model_path}. Using untrained model for demo.")
    except Exception as e:
        print(f"Error loading model: {e}. Using untrained model for demo.")
    
    # Ensure model is on CPU for inference service
    model = model.cpu()
    model.eval()
    print(f"Model moved to CPU for inference service compatibility")
    return model

# Load the model
trained_model = load_trained_model()

# Create sample input for model signature
sample_data = session.table("CIFAR10_TRAINING_DATA").limit(1).to_pandas()
sample_input = session.create_dataframe(sample_data[['IMAGE_DATA']])

# Create model instance with context
cifar10_model = CIFAR10ClassificationModel(
    context=custom_model.ModelContext(
        models={'cifar10_classifier': trained_model}
    )
)

print("Model prepared for registration!")


In [ ]:
# Initialize Model Registry
ml_registry = Registry(session=session)

# Register the model
try:
    model_version = ml_registry.log_model(
        cifar10_model,
        model_name="CIFAR10_CLASSIFIER",
        version_name="v2_distributed",
        sample_input_data=sample_input,
        target_platforms = ['SNOWPARK_CONTAINER_SERVICES'],
        conda_dependencies=["pytorch", "torchvision", "pillow", "numpy"],
        options={
            "embed_local_ml_library": True,
            "relax": True
        }
    )
    
    print(f"Model registered successfully!")
    print(f"Model name: CIFAR10_CLASSIFIER")
    print(f"Version: v1_distributed")
    print(f"Model version object: {model_version}")
    
except Exception as e:
    print(f"Error registering model: {e}")
    print("This might be due to missing model file or registry permissions.")


In [ ]:
# Direct inference using the trained model in notebook environment
try:
    # Get test data
    test_data = session.table("CIFAR10_TRAINING_DATA").limit(3).to_pandas()
    
    print("Running direct inference on test samples...")
    
    # Use the trained model directly
    trained_model.eval()
    
    # Process test images
    results = []
    for i, row in test_data.iterrows():
        # Decode and transform image
        base64_image = row['IMAGE_DATA']
        image_data = base64.b64decode(base64_image)
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        
        # Apply transforms
        transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
        
        # Make prediction
        with torch.no_grad():
            outputs = trained_model(image_tensor)
            probabilities = F.softmax(outputs, dim=1)
            predicted_class = torch.argmax(outputs, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        results.append({
            'actual_label': row['CLASS_NAME'],
            'predicted_class': predicted_class,
            'predicted_label': cifar10_classes[predicted_class],
            'confidence': confidence
        })
    
    print("\nDirect Inference Results:")
    for i, result in enumerate(results):
        print(f"Sample {i+1}:")
        print(f"  Actual: {result['actual_label']}")
        print(f"  Predicted: {result['predicted_label']}")
        print(f"  Confidence: {result['confidence']:.4f}")
        print()
    
except Exception as e:
    print(f"Error during direct inference: {e}")
    import traceback
    traceback.print_exc()
